In [1]:
from sklearn.model_selection import train_test_split
from src.baseline_experiments import *
import pandas as pd
import sklearn
import numpy as np
import glob
from mountaineer import Mountaineer
from gale import create_mapper

## Import Diabetes dataset

In [2]:
np.random.seed(42)
torch.manual_seed(42)

df = pd.read_csv("./dataset/diabetes.csv")
df.drop(columns=["p_id"], inplace=True)

In [3]:
X = df.drop(columns=["diabetes"])
y = df["diabetes"]

## Train a neural network model with two hidden layers

In [4]:
batch_size = 16
num_epochs = 80
learning_rate = 4e-3
N = 100
shap_sample_size = 10
#possible blur/mean/zero
imputation_typ = 'blur'


In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X_train_tens = torch.tensor(X_train.to_numpy()).float()
X_test_tens = torch.tensor(X_test.to_numpy()).float()

In [6]:
unique, counts = np.unique(y_train, return_counts=True)
    
averaging = 'binary'
y_train_tens = torch.tensor(y_train.to_numpy()).view(-1, 1).float()
output_dim = 1

In [7]:
device = torch.device("cpu")

dataset_tens = torch.utils.data.TensorDataset(X_train_tens, y_train_tens)
train_iter = torch.utils.data.DataLoader(dataset_tens, batch_size, shuffle=False)

print("Try to load the model..")
model = nn_model.get_model(device, train_iter, X_train.shape[1], output_dim, averaging, learning_rate, num_epochs)

Try to load the model..
Train model
Epoch 020: | Loss: 0.30523 | Acc: 86.654 | F1: 79.20633
Epoch 040: | Loss: 0.28288 | Acc: 90.462 | F1: 85.07238
Epoch 060: | Loss: 0.20930 | Acc: 92.577 | F1: 88.17563
Epoch 080: | Loss: 0.18856 | Acc: 92.077 | F1: 87.61079
Set to eval


In [8]:
X_test_tens = torch.tensor(X_test.to_numpy()).float()

X_test_c, Y_test_c = get_correct_predictions(model,X_test.reset_index(drop=True),X_test_tens,y_test.reset_index(drop=True),averaging)
X_test_c_tens = torch.from_numpy(X_test_c.to_numpy(dtype=np.float32))

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

## Use the sigmoid function as the prediction probability function for Mapper

In [ ]:
predictions = torch.sigmoid(model(X_test_tens)).detach().numpy()
plt.plot(np.sort(predictions.flatten()))

## Compute the Explanations of (IntGrad, SHAP, DeepLIFT) $\times$ (Zero, Max Distances, Uniform, Gaussian, Train)-Baselines (15 explanation outputs in total)


In [ ]:
print("Loading all baselines..")
bun = BaselineUtilTensor()
black_baseline = bun.create_black_baseline(X)
uniform_baseline = bun.create_uniform_baseline(X)
gaussian_baseline = bun.create_gaussian_baseline(X, 0.5)
train_baseline = bun.create_train_baseline(X)

In [ ]:
explanations = {}
methods = ['IG','DeepSHAP','DeepLIFT']
target = None
for method in methods:
    bb_attr, mdb_attr, ub_attr, gb_attr, tb_attr = get_attr_scores(method, X_test_tens, None, black_baseline,uniform_baseline,gaussian_baseline, train_baseline, model,X.min(), X.max(), X.columns)
    explanations[method] = [bb_attr, mdb_attr, ub_attr, gb_attr, tb_attr]
    #for exp in explanations[method]:
        #exp[exp>5]=5
        #exp[exp<-5]=-5

In [ ]:
#baselines = ['zero', 'max distance', 'uniform', 'gaussian', 'trained']

## Creating the mapper outputs

In [ ]:
import sklearn
mapper1 = create_mapper(explanations['IG'][0], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper2 = create_mapper(explanations['IG'][1], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper3 = create_mapper(explanations['IG'][2], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper4 = create_mapper(explanations['IG'][3], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper5 = create_mapper(explanations['IG'][4], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)

mapper6 = create_mapper(explanations['DeepSHAP'][0], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper7 = create_mapper(explanations['DeepSHAP'][1], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper8 = create_mapper(explanations['DeepSHAP'][2], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper9 = create_mapper(explanations['DeepSHAP'][3], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper10 = create_mapper(explanations['DeepSHAP'][4], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)


mapper11 = create_mapper(explanations['DeepLIFT'][0], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper12 = create_mapper(explanations['DeepLIFT'][1], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper13 = create_mapper(explanations['DeepLIFT'][2], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper14 = create_mapper(explanations['DeepLIFT'][3], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)
mapper15 = create_mapper(explanations['DeepLIFT'][4], np.array([np.squeeze(i) for i in predictions]), resolution=10, gain=0.4, dist_thresh=0.5)


In [9]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:1920px !important; }</style>"))

In [10]:

#list of mapper outputs - minimum 2
mapper_outputs=[mapper1, mapper2, mapper3, mapper4, mapper5, mapper6, mapper7, mapper8, mapper9, mapper10, mapper11, mapper12, mapper13, mapper14, mapper15]

explanation_vectors=[explanations['IG'][0],explanations['IG'][1],explanations['IG'][2],explanations['IG'][3],explanations['IG'][4],  explanations['DeepSHAP'][0], explanations['DeepSHAP'][1], explanations['DeepSHAP'][2], explanations['DeepSHAP'][3], explanations['DeepSHAP'][4], explanations['DeepLIFT'][0], explanations['DeepLIFT'][1], explanations['DeepLIFT'][2], explanations['DeepLIFT'][3], explanations['DeepLIFT'][4]]
explanation_list=[]
for expl in explanation_vectors:
    explanation_list.append(expl.tolist())
predicted_prob = np.array([np.squeeze(i) for i in predictions])
expl_labels=['IG_ZB', 'IG_MB', 'IG_UB', 'IG_GB', 'IG_TB', 'DS_ZB', 'DS_MB', 'DS_UB', 'DS_GB', 'DS_TB', 'DL_ZB', 'DL_MB', 'DL_UB', 'DL_GB', 'DL_TB'] 

#column names of the dataframe
column_names= np.array(X.columns)
class_labels = {1:"Diabetic", 0:"Non Diabetic"}


NameError: name 'mapper1' is not defined

In [11]:
#visualize
mnt = Mountaineer()
mnt.visualize(X_test.to_numpy(),y_test.to_numpy(), predicted_prob, explanation_list, mapper_outputs, column_names, expl_labels, class_labels) #Projection method- 'UMAP' or 'TSNE'

NameError: name 'predicted_prob' is not defined

In [17]:
#mean of explanation attributions for every feature zero baseline Integrated Gradient
np.max(explanation_vectors[0], axis=0)

array([  88.29104764,   49.91734716,  797.81816227,  175.17331936,
        769.54749645,  389.75980937,    6.34245809, 1554.15616189])

In [18]:
#means of explanation attributions for every feature for max distance baseline Integrated Gradients
np.mean(explanation_vectors[1], axis=0)

array([ 8492.13286346, 14595.81531351,  6431.98618104, -4799.87846332,
        3101.18180703, -2723.83870671,  1184.22380482, 15918.78830806])

In [19]:
#mean of explanation attributions for every feature zero baseline Integrated Gradient
np.median(explanation_vectors[0], axis=0)

array([  1.6603706 ,  -9.0945436 ,   0.        ,  -3.46607467,
       -10.6932722 ,   0.        ,   0.88227119,   6.25249074])

In [20]:
np.min(explanation_vectors[0], axis=0)

array([  -17.46102124, -1068.11921051,  -277.73046875,  -173.98871572,
        -144.92184172,  -432.33702606,   -29.68376723,  -424.4705105 ])

In [21]:
X_test.describe()

,no_times_pregnant,glucose_concentration,blood_pressure,skin_fold_thickness,serum_insulin,bmi,diabetes pedigree,age
count,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000,203.000000
mean,4.019704,119.832512,68.857143,21.024631,84.177340,31.002956,0.451823,34.802956
std,3.422350,32.489473,20.245383,15.851551,121.087719,8.567547,0.279240,12.898714
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.085000,21.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,26.800000,0.255000,24.000000
50%,3.000000,117.000000,72.000000,23.000000,48.000000,31.200000,0.361000,31.000000
75%,6.000000,138.000000,80.000000,32.000000,131.000000,36.000000,0.629500,42.500000
max,17.000000,196.000000,104.000000,54.000000,846.000000,49.700000,1.731000,81.000000
